In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 35801695
paper_name = 'stenberg_warringer_2022' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/Figure 1-figure supplement 2 source data 1.xlsx', sheet_name='Data S8 - Deletion collection')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4580 x 7


In [7]:
original_data.head()

,Environment,ORF,Gene,N,Generation time Dnorm,SD Dnorm,SEM Dnorm
0,Paraquat,YAL002W,VPS8,6,0.477844,0.099148,0.040477
1,Paraquat,YAL004W,NaN,6,0.288856,0.074057,0.030234
2,Paraquat,YAL005C,SSA1,6,0.450347,0.056721,0.023156
3,Paraquat,YAL007C,ERP2,6,0.021934,0.092350,0.037702
4,Paraquat,YAL008W,FUN14,6,0.314813,0.129227,0.052757


In [8]:
original_data['orf'] = original_data['ORF'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Environment, ORF, Gene, N, Generation time Dnorm, SD Dnorm, SEM Dnorm, orf]
Index: []


In [12]:
original_data['data'] = original_data['Generation time Dnorm']

In [13]:
original_data.set_index('orf', inplace=True)

In [14]:
original_data = original_data[['data']].copy()

In [15]:
original_data = original_data.groupby(original_data.index).mean()

In [16]:
original_data.shape

(4555, 1)

In [19]:
# Higher doubling time = slower growth (need to convert for convention)
original_data['data'] = -original_data['data']

# Prepare the final dataset

In [20]:
data = original_data.copy()

In [21]:
dataset_ids = [22267]
datasets = datasets.reindex(index=dataset_ids)

In [22]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [23]:
data.head()

dataset_id,22267
data_type,value
orf,
YAL002W,-0.477844
YAL004W,-0.288856
YAL005C,-0.450347
YAL007C,-0.021934
YAL008W,-0.314813


## Subset to the genes currently in SGD

In [24]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [25]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,22267
,data_type,value
gene_id,orf,
2,YAL002W,-0.477844
1863,YAL004W,-0.288856
4,YAL005C,-0.450347
5,YAL007C,-0.021934
6,YAL008W,-0.314813


# Normalize

In [26]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [27]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [28]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id          22267          
data_type           value    valuez
gene_id orf                        
2       YAL002W -0.477844 -0.945038
1863    YAL004W -0.288856 -0.527266
4       YAL005C -0.450347 -0.884253
5       YAL007C -0.021934  0.062786
6       YAL008W -0.314813 -0.584647

# Print out

In [29]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')